# Libraries

In [1]:
import numpy as np
import sympy as sp
import sympy.physics.quantum as sq
import sympy.physics.quantum.qubit as sqq
import sympy.physics.quantum.gate as sqg

import cvxpy as cp

import matplotlib.pyplot as plt
import networkx as nx

sp.init_printing(use_latex='mathjax')
from IPython.display import display

## Extending the braket library to include orthogonality

In [2]:
class onStruct:
    G = nx.Graph()

class onKet(sq.Ket,onStruct):
    def _eval_innerproduct(self,bra,**hints):
        #Check if the innerproduct is with yourself, then set 1
        if sq.Dagger(bra) == self:
            return sp.Integer(1)
        #Else, if the innerproduct is with a neighbor from the graph, set 0
        elif sq.Dagger(bra) in G[self]:
            return sp.Integer(0)
        #Else, do nothing; not enough information
    
    @classmethod
    def set_on_rel(cls,givenG):
        cls.G=givenG
    
    @classmethod
    def dual_class(self):
        return onBra

    
class onBra(sq.Bra,onStruct):
    @classmethod
    def dual_class(self):
        return onKet

In [3]:
class bKet(sq.Ket):
    
    #total number of kets (automatically increases as new instances are initialised)
    totalKets=0
    
    #this method automates the increase
    @classmethod
    def _eval_args(cls,args):
        #validate input type
        if not isinstance(args[0],int):
            raise ValueError("Integer expected in the argument, got: %r"%args[0])

        #if a ket |5> is initialised, the dimension is assumed to be 6 since we have |0>,|1>,...|5>
        if int(args[0])+1 > cls.totalKets:
            cls.totalKets=args[0]+1
        
        #pass control back to the base class
        return sq.Ket._eval_args(args) 
    
    #Could not find a way of using just one function; need to go through the two
    #based on a combination of the qubit implementation and the 
    def _represent_default_basis(s,**options):
        return s._represent_bKet(None,**options) #syntax taken from the Qubit library
    
    def _represent_bKet(s,basis,**options): #_represent_default_basis
        a=np.zeros(s.__class__.totalKets,dtype=int)
        a[s.label[0]]=1
        return sp.Matrix(a)
    
    @classmethod
    def set_dimension(cls,arg):
        cls.totalKets=arg
    
    def _eval_innerproduct(self,bra,**hints):
        #Check if the innerproduct is with yourself, then set 1
        if sq.Dagger(bra) == self:
            return sp.Integer(1)
        #Else, if the innerproduct is with a neighbor from the graph, set 0
        else:
            return sp.Integer(0)
        #Else, do nothing; not enough information
    

    @classmethod
    def dual_class(self):
        return bBra

    
class bBra(sq.Bra):
    # @classmethod
    # def _eval_args(cls,args):
    #     return args
    
    @classmethod
    def dual_class(cls):
        return bKet

### Debugging | representation example

In [ ]:
class SzUpKet(sq.Ket):
    def _represent_SzUpKet(s,basis,**options):
        return sp.Matrix([1,0])
    
#     def _represent_SzOp(s,basis,**options):
#         return sp.Matrix([1,0])

class SzOp(sq.Operator):
    pass

sz = SzOp('Sz')
up = SzUpKet('up')
sq.represent(up,basis=SzUpKet) #,basis=sz)

### Testing | Dimension increasing?

In [ ]:
# class bKet(sq.Ket):
    
#     #total number of kets (automatically increases as new instances are initialised)
#     totalKets=0
    
#     #this method automates the increase
#     @classmethod
#     def _eval_args(cls,args):
#         #validate input type
#         if not isinstance(args[0],int):
#             raise ValueError("Integer expected in the argument, got: %r"%args[0])

#         #if a ket |5> is initialised, the dimension is assumed to be 6 since we have |0>,|1>,...|5>
#         if int(args[0])+1 > cls.totalKets:
#             cls.totalKets=args[0]+1
        
#         #pass control back to the base class
#         return sq.Ket._eval_args(args) 
    
#     #Could not find a way of using just one function; need to go through the two
#     #based on a combination of the qubit implementation and the 
#     def _represent_default_basis(s,**options):
#         return s._represent_bKet(None,**options) #syntax taken from the Qubit library
    
#     def _represent_bKet(s,basis,**options): #_represent_default_basis
#         a=np.zeros(s.__class__.totalKets,dtype=int)
#         a[s.label[0]]=1
#         return sp.Matrix(a)
    
#     @classmethod
#     def set_dimension(cls,arg):
#         cls.totalKets=arg
    
#     def _eval_innerproduct(self,bra,**hints):
#         #Check if the innerproduct is with yourself, then set 1
#         if sq.Dagger(bra) == self:
#             return sp.Integer(1)
#         #Else, if the innerproduct is with a neighbor from the graph, set 0
#         else:
#             return sp.Integer(0)
#         #Else, do nothing; not enough information
    

#     @classmethod
#     def dual_class(self):
#         return bBra

    
# class bBra(sq.Bra):
#     # @classmethod
#     # def _eval_args(cls,args):
#     #     return args
    
#     @classmethod
#     def dual_class(cls):
#         return bKet

In [ ]:
a=bKet(0)

b=a*sq.Dagger(a)

print(bKet.totalKets)
sq.represent(a)

c=bKet(3);
sq.represent(a)
sq.represent(b)
sq.represent(c)

In [ ]:
a=bKet(0); a

In [ ]:
sq.represent(a)

In [ ]:
b=bKet(1); b

In [ ]:
sq.represent(b)

In [ ]:
sq.represent(a)

In [ ]:
c=bKet(3); c

In [ ]:
sq.represent(a)

## Extending the tensor product functionality—tsimp

In [20]:
def powerDrop(expr):
    if isinstance(expr,sp.Pow): #TODO: make sure the base is not too complex
        # print("PowerEncountered")
        if expr.exp>=2:
            # print("glaba")
            # display(expr.base)
            return sq.qapply(sp.Mul(expr.base,expr.base))*sp.Pow(expr.base,expr.exp-2)
        else:
            return expr #autoDropDim(sp.Mul(expr.base,expr.base))
    else:
        if expr.has(sp.Pow):
            #if it is a sum or a product, run this function for each part and then combine the parts; return the result
            if isinstance(expr,sp.Mul) or isinstance(expr,sp.Add):
                new_args=[] #list(expr.args)
                for _ in expr.args:
                    new_args.append(powerDrop(_))
                if isinstance(expr,sp.Mul):        
                    return sp.Mul(*new_args)
                elif isinstance(expr,sp.Add):
                    return sp.Add(*new_args)  
                
            #There would be no else here because tensor product simp would have removed that part
        else:
            return expr        
    
def autoDropDim(expr):
    #print("Expression")
    #if isinstance(expr,sp.Mul):
        #print("type:multiplier")
    #display(expr)
    
    
    if isinstance(expr,sq.TensorProduct):
        new_args=[]
        for _ in expr.args:
            #display(_)
            #print(type(_))
            if _ != sp.Integer(1):
            #if not isinstance(_,core.numbers.One):
                new_args.append(_)
        #print("TensorProduct with %d non-ones in the tensor product"%len(new_args))
        if(len(new_args)==0):
            return sp.Integer(1)
        else:
            return sq.TensorProduct(*new_args)
    else:
        if expr.has(sq.TensorProduct):
            #if it is a sum or a product, run this function for each part and then combine the parts; return the result
            if isinstance(expr,sp.Mul) or isinstance(expr,sp.Add):
                new_args=[] #list(expr.args)
                for _ in expr.args:
                    new_args.append(autoDropDim(_))
                if isinstance(expr,sp.Mul):        
                    return sp.Mul(*new_args)
                elif isinstance(expr,sp.Add):
                    return sp.Add(*new_args)  
                
            #There would be no else here because tensor product simp would have removed that part
        else:
            return expr #when the expression is just an integer or some such


def tsimp(e):
    return autoDropDim(sq.tensorproduct.tensor_product_simp(sq.qapply(e)).doit())
    #return autoDropDim(sq.tensorproduct.tensor_product_simp_Mul(e).doit())
    #return autoDropDim(sq.tensorproduct.tensor_product_simp_Mul(sq.qapply(e)).doit())
    #return autoDropDim(sq.tensorproduct.tensor_product_simp(e).doit())

# Swap KCBS | Playfield 

## Inputs

In [ ]:
class inputs:
    graphSize = 5
    p = 1/np.sqrt(5)*np.array([1,1,1,1,1])

## The Sympy Part | playground

In [ ]:
k0 = sq.Ket('φ')
k1 = sq.Ket('ψ')
superposition = k0+k1; superposition

In [ ]:
inner_prod = sq.Dagger(superposition)*superposition; inner_prod

In [ ]:
sq.qapply(inner_prod)

In [ ]:
density = sq.qapply(superposition*sq.Dagger(superposition)); density

In [ ]:
qubit_state = sqq.Qubit('00'); qubit_state

In [ ]:
isinstance(qubit_state,sq.State)

In [ ]:
gate_operator = sqg.HadamardGate(0); gate_operator

In [ ]:
isinstance(gate_operator,sq.Operator)

In [ ]:

a=sqq.Qubit(0)
b=sqq.Qubit(1)

d = sq.qapply(sq.Dagger(a)*b).doit(); d

### Playing with classes

In [ ]:
class A:
    gla=1

class B:
    glaba=3
    
class C(B,A):
    glow=4

c=C()

print(isinstance(c,A),isinstance(c,B),isinstance(c,C))


### Kishor meets the class

In [ ]:
class Person:
    name="blank"
    
    def set_name(self,arg):
        self.name = arg
    
    @property
    def myName(self):
        return self.name

    def get_name(self):
        return self.name
    
Kishor = Person() 
Kishor.set_name("Bharti")

M = Person()
M.set_name("M")



print(M.myName, Kishor.myName, M.get_name() )

print(M.myName,Kishor.myName)

In [ ]:
class Person:
    name="blank"
    compatibility=""
    
    def set_name(self,arg):
        self.name = arg
    
    @property
    def myName(self):
        return self.name
    
    @classmethod
    def set_comp(cls,arg):
        cls.compatibility=arg
        
    @classmethod
    def get_comp(cls):
        return cls.compatibility
        
    @property
    def comp(self):
        return self.compatibility
    
Kishor = Person() 
Kishor.set_name("Bharti")

M = Person()
M.set_name("M")

Person.set_comp("M hates K")
print(Person.get_comp())

print(M.myName,Kishor.myName)

print(M.comp,Kishor.comp)

In [ ]:
class A:
    aVal="blank"
    
    @property
    def myVal(self):
        return self.aVal

class C:
    cVal="blank3"
    
    @property
    def myVal3(self):
        return self.cVal
    
class B(A,C):
    bVal="blank2"
    
    @property
    def myVal2(self):
        return self.bVal
    
a = B()

print(a.myVal2,a.myVal,a.myVal3)

### Preliminary implementation of orthogonormal kets (uses labels)

In [ ]:
class onKet(sq.Ket):
#     a=1
    
    #@classmethod
    #this @classmethod descriptor was killing it
    #it was sending the class in "self" instead of the instance in "self"
    #makes sense now
    #I can perhaps use this to set orthogonality relations a little more generally
    def _eval_innerproduct(self,bra,**hints):
        #print(self.label,bra.label)
        if self.label == bra.label:
            return sp.Integer(1)
        else:
            return sp.Integer(0)
    
    @classmethod
    def dual_class(self):
        return onBra
    @property
    def iAm(self):
        return self.label
    
class onBra(sq.Bra):
#     a=2

#     @classmethod
#     def _eval_innerproduct(self,ket,**hints):
#         if self.label == ket.label:
#             return sp.Integer(1)
#         else:
#             return sp.Integer(0)
    
    @classmethod
    def dual_class(self):
        return onKet

In [ ]:
a=onKet(0)
b=onKet(1)

In [ ]:
c=a+b; c

In [ ]:
sq.qapply(sq.Dagger(b)*a).doit()

In [ ]:
d=sq.Dagger(c)*b
sq.qapply(d).doit()

### Orthogonal Ket, general orthogonality relation

In [ ]:
class onStruct:
    G = nx.Graph()

class onKet(sq.Ket,onStruct):
#     a=1
    #@classmethod
    #this @classmethod descriptor was killing it
    #it was sending the class in "self" instead of the instance in "self"
    #makes sense now
    #I can perhaps use this to set orthogonality relations a little more generally
    def _eval_innerproduct(self,bra,**hints):
        #print(self.label,bra.label)        
        if sq.Dagger(bra) == self:
        #if self.label == bra.label:
            return sp.Integer(1)
        elif sq.Dagger(bra) in G[self]:
            return sp.Integer(0)
    
    @classmethod
    def set_on_rel(cls,givenG):
        cls.G=givenG
    
    @classmethod
    def dual_class(self):
        return onBra

    
class onBra(sq.Bra,onStruct):
#     a=2

#     @classmethod
#     def _eval_innerproduct(self,ket,**hints):
#         if self.label == ket.label:
#             return sp.Integer(1)
#         else:
#             return sp.Integer(0)    
    @classmethod
    def dual_class(self):
        return onKet

In [ ]:
#define the kets
a=onKet(1)
b=onKet(2)
c=onKet(3)
d=onKet(4)

#define the orthogonality relation using a graph (edges mean orthogonal)
G = nx.Graph()
G.add_nodes_from([a,b,c,d])
G.add_edges_from([[a,b],[b,c],[c,d],[d,a]])

#Add this graph to the class
onKet.set_on_rel(G)

#test
s1 = (a+b+c)/sp.sqrt(3)
s2 = (b+c+d)/sp.sqrt(3)

d = sq.Dagger(s1)*s2

#simplify
sq.qapply(d).doit()

In [ ]:
#plot the graph for fun
nx.draw(G, with_labels=True)

In [ ]:
f=sq.Ket("f")
g=sq.Ket("g")
h=sq.TensorProduct(a,b)
l=sq.TensorProduct(b,a)
sq.qapply(sq.Dagger(h)*l).doit()

### Playing with networks

In [ ]:
G = nx.Graph()
G.add_node(1)

In [ ]:
G = nx.Graph()
G.add_nodes_from([1,2,3,4,5])
G.add_edges_from([[1,2],[2,3],[3,4],[4,5],[5,1]])

In [ ]:
%matplotlib inline 
nx.draw(G, with_labels=True)

In [ ]:
3 in G[1]

# Ideal KCBS calculations

## Working Version

In [21]:
class cls_iKCBS:    
    
    def __init__(s,n):
        cls_iKCBS.n=n
        #j in 0,1,2,3,4
        s.ke = [bKet(j) for j in range(3)]    
        s.br = [sq.Dagger(s.ke[j]) for j in range(3)]
        s.Πs = []
        
        #evaluate the projectors for the ideal case
        for j in range(n):
            s.Πs.append(s.eval(j,n))
        
        #calculate the matrix representation
        s.matΠs = [sq.represent(_) for _ in s.Πs]
        
        #the first two vectors are taken to be k'
        s.kp = [s.eval(0,n,True),s.eval(1,n,True)]
        
        #the third is constructed as k0 x k1 (cross product) and then added to the list
        s.kp.append(s.cross(s.kp))
        
        #evaluates the conjugate for each element in the list
        s.bp=[sq.Dagger(ikp) for ikp in s.kp]                
        
        #the Translation operator
        s.T = s.kp[0]*s.bp[2] + s.kp[1]*s.bp[0] + s.kp[2]*s.bp[1] 
        s.Ti = s.kp[2]*s.bp[0] + s.kp[0]*s.bp[1] + s.kp[1]*s.bp[2] #TODO: Automate this
        
        s.matT=sq.represent(s.T)
        
        #Now the abstract ones
        
        #creates n abstract projectors (basically just a bunch of non-commuting symbols)
        s._Πs = sp.symbols('Π_0:%d'%n,commutative=False)
        
        #An abstract psi (likely we won't use this)
        #s._Ψ = sp.symbol('Psi')
        
        #the following will find _Τ as a function of _Πs 
        s._T=s.eval_c() #NB: it computes the coefficients into s.c
        
#         #Just trying (original)
#         s.TP = sq.TensorProduct #just to make naming easy
#         s._U_=s.TP(sq.IdentityOperator(),s.kp[0]*s.bp[0]) + s.TP(s._T,s.kp[1]*s.bp[1]) + s.TP(s._T,s.kp[2]*s.bp[2])        
#         s._T_=s.TP(sq.IdentityOperator(),s.kp[0]*s.bp[0] + s.kp[2]*s.bp[1] + s.kp[1]*s.bp[2])
#         s._V_=s.TP(s._Πs[0],sq.IdentityOperator()) + s.TP(s._Πs[1],s.Ti) + s.TP(s._Πs[2],s.Ti*s.Ti)


#         s.S = s._T_ * s._U_ * s._V_ #* s._U_
#         s._ρin_ = s.TP(sq.IdentityOperator(),s.ke[0]*s.br[0])
#         s._kout_ = s.TP(sq.IdentityOperator(),s.kp[0])
#         s._bout_ = s.TP(sq.IdentityOperator(),s.bp[0])

#         s.F = (s._bout_ * s.S * s._ρin_ * sq.Dagger(s.S) * s._kout_)

        #Just trying (debugging)
        s.TP = sq.TensorProduct #just to make naming easy
        s._U_=s.TP(sp.Integer(1),s.kp[0]*s.bp[0]) 
        s._T_=s.TP(sp.Integer(1),s.kp[0]*s.bp[0])
        s._V_=s.TP(s._Πs[0],sp.Integer(1))
        
        s.S = s._T_ * s._U_ * s._V_ #* s._U_
        s._ρin_ = s.TP(sp.Integer(1),s.ke[0]*s.br[0])
        s._kout_ = s.TP(sp.Integer(1),s.kp[0])
        s._bout_ = s.TP(sp.Integer(1),s.bp[0])        

        #s.F2=tsimp(s._bout_ * tsimp(tsimp(s._T_*s._U_)*s._V_)*s._kout_)
        s.F2=tsimp(s._T_*s._U_)
        #s.F3=s.F2 * s._V_
        
        s.F1 = s._bout_ * s._T_ * s._U_ * s._V_ * s._kout_
        #s.F2 = s.F1 * 
    
    def cross(s,vecs):
        ca=[sq.qapply(sq.Dagger(s.ke[i])*vecs[0]).doit() for i in range(3)]
        cb=[sq.qapply(sq.Dagger(s.ke[i])*vecs[1]).doit() for i in range(3)]
        
        res = (ca[2-1]*cb[3-1] - ca[3-1]*cb[2-1])*s.ke[0] + \
             (ca[3-1]*cb[1-1]-ca[1-1]*cb[3-1])*s.ke[1] + \
             (ca[1-1]*cb[2-1] - ca[2-1]*cb[1-1])*s.ke[2]
        
        return res

    #evaluates the matrices and vectors
    def eval(s,j,n,ve=False):
        N=sp.Integer(n)
        J=sp.Integer(j)
        #print(N,j)
        one=sp.Integer(1)
        #print(one)
        α1=sp.cos((sp.pi)/N)/(one+sp.cos(sp.pi/N)) #verified
        #a1=sp.cos(sp.pi/N)
        #
        #print(α1)
        
        α2=one-α1 #verified
        α3=J * sp.pi * (N-one)/N #verified
        
        vec = sp.sqrt(α1)*s.ke[0] + sp.sqrt(α2)*sp.sin(α3)*s.ke[1] + sp.sqrt(α2)*sp.cos(α3)*s.ke[2]
        
        projector = vec*sq.Dagger(vec)
           
        #matrixprojector = sp.Matrix([0,0])
        #return projector
        
        if ve==False:
            return projector
        else:
            return vec

    
    def eval_c(s,varCount=None):
        cls=s.__class__
        if varCount==None:
            varCount=cls_iKCBS.n*2
        if varCount <= cls.n:
            raise ValueError("number of variables should be at least n")
    
        a=sp.symbols('a0:%d'%varCount)
        #s.b=sp.symbols('b0:%d'%varCount)
        b=[] #family of solutions
        s.c=[] #particular solution with the free variables set to one
        
        #Setting up of constraints
        #If the system is not able to find a solution, consider increasing
        #the number of variables by taking different sets of products or products of three operators
        TfΠ=a[0]*s.Πs[0]
        _TfΠ=a[0]*s._Πs[0]
        for i in range(1,cls.n):
            TfΠ=TfΠ + a[i]*s.Πs[i]
            _TfΠ=_TfΠ + a[i]*s._Πs[i]
        for i in range(cls.n,varCount):
            TfΠ=TfΠ + a[i]*s.Πs[i%cls.n]*s.Πs[(i+2)%cls.n]
            _TfΠ=_TfΠ + a[i]*s._Πs[i%cls.n]*s._Πs[(i+2)%cls.n]
            
            
        LHS=sq.represent(TfΠ)
        
        # LHS=a[0]*s.matΠs[0]
        # for i in range(1,cls.n):
        #     LHS=LHS+a[i]*s.matΠs[i]
        # for i in range(cls.n,varCount):
        #     LHS=LHS+a[i]*s.matΠs[i%cls.n]*s.matΠs[(i+2)%cls.n]
            
        RHS=s.matT
        
        soln=sp.solve(sp.N(LHS-RHS),a,dict=True)

        '''
            #soln=sp.solve([a[0]-1,a[1]-2,a[2]-3,a[3]-4,a[4]-5,a[5]-1,a[6]-1,a[7]-1,a[8]-1,a[9]-1],a,dict=True)

            # sols = solve([t1 + t2 + t3, eq1, eq2], [t1, t2, t3], dict=True)
            # sols[0][t1] # This is t1 in the first solution        
        '''

        #Dictionary to assign one/zero to free variables
        dRem={}
        
        #IMPORTANT: 
        #In sympy, symbols/expressions are unmutable (you can't change them ever);
        #You can save a substituted expression into a new expression (and use a symbol to denote it)
        #To wit: when you substitute, a new expression is produced

        for i in range(varCount):
            #the variables which were evaluated to an expression, assign them to b[i]
            try:
                b.append(soln[0][a[i]])      
            #if the variable was free, assign b[i] to be one
            #and create a dictionary to substitute these free variables
            except:
                b.append(sp.Integer(1))
                dRem[a[i]]=1
        
        #in the solution saved into b[i], substitute for the free variables using the dictionary
        #save the result into the variable c
        s.c = [_.subs(dRem) for _ in b]

        #substitute the solution into the coefficients in Tfπ_p, the operator T as a sum of projectors (and its products)
        dFin={}
        for i in range(varCount):
            dFin[a[i]]=s.c[i]                
        
        return _TfΠ.subs(dFin)
    
    
    #This doesn't work as expected
    def partial_trace(s,M):
        
        s.br_ = [ sq.TensorProduct(sq.IdentityOperator(),_) for _ in s.br]
        s.ke_ = [ sq.TensorProduct(sq.IdentityOperator(),_) for _ in s.ke]
        
        res=0*s.br_[0]*M*s.ke_[0]        #to get the type right!
        for i in range(3):
            for j in range(3):
                res=res+s.br_[i]*M*s.ke_[j]
        #return sq.qapply(res).doit()
        return tsimp(res)
        

In [18]:
iKCBS = cls_iKCBS(5)

In [22]:
res2=iKCBS.F2 #powerDrop(iKCBS.F2)
powow=iKCBS.F2.args[0]
display(powow)
print(isinstance(powow,sp.Pow))
print(powow.exp)
display(sq.qapply(powow.base*powow.base))

display(powerDrop(iKCBS.F2))


#display(res2)

         2
(❘2⟩⋅⟨2❘) 

True
2


❘2⟩⟨2❘

                              ________       _______________________________  
                             ╱ 1   √5       ╱       √5           1            
                            ╱  ─ + ── ⋅    ╱  - ────────── - ────────── + 1 ⋅❘
                          ╲╱   4   4      ╱       ⎛√5   5⎞     ⎛√5   5⎞       
                                         ╱      4⋅⎜── + ─⎟   4⋅⎜── + ─⎟       
  ❘0⟩⟨0❘     √5⋅❘0⟩⟨0❘                 ╲╱         ⎝4    4⎠     ⎝4    4⎠       
────────── + ────────── + ────────────────────────────────────────────────────
  ⎛√5   5⎞     ⎛√5   5⎞                              ________                 
4⋅⎜── + ─⎟   4⋅⎜── + ─⎟                             ╱ √5   5                  
  ⎝4    4⎠     ⎝4    4⎠                            ╱  ── + ─                  
                                                 ╲╱   4    4                  

            ________       _______________________________                    
           ╱ 1   √5       ╱       √5           1   

In [ ]:
res=iKCBS._T_ * iKCBS._U_
display(res)
display(tsimp(res))

In [ ]:
display(iKCBS._U_)

In [ ]:
display(iKCBS._V_)

In [ ]:
display(iKCBS._T_)

In [ ]:
display(iKCBS.F1)

display(tsimp(iKCBS.F1))
#sq.qapply(iKCBS.F1).doit()

In [ ]:

iKCBS=cls_iKCBS(5)

#sp.init_printing()
#sp.pprint(iKCBS.c[0])
for i in range(10):
    display(iKCBS.c[i])
    

display(sp.N(iKCBS.matΠs[1]*iKCBS.matΠs[3]))
display(sp.N(sq.represent(iKCBS.T)))
display(iKCBS._T)
#display(sp.N(iKCBS._T_))
#display(sp.N(iKCBS._U_))
#display(sp.N(iKCBS._V_))
display(sp.N(iKCBS.F))

In [ ]:
#display(iKCBS._U_)

#display(iKCBS.partial_trace(iKCBS._U_))



In [ ]:
g = bKet(0)

Id=sq.IdentityOperator()

display(Id*g)

del Id
del g

In [249]:
g = bKet(0)
h = bKet(1)

TP=sq.TensorProduct
zz=TP(g,g)
zo=TP(g,h)
oo=TP(h,h)
oz=TP(h,g)

display(oz)

ρ = 0.5 * oz*sq.Dagger(oz) + 0.4*oo*sq.Dagger(oo) + 0.1*oz*sq.Dagger(oo)

display(ρ)

display(tsimp(sq.Dagger(oz)*ρ*oz))

display(sq.qapply(sq.Dagger(oz)*ρ*oz))

del TP

❘1⟩⨂ ❘0⟩

0.5⋅❘1⟩⨂ ❘0⟩⋅⟨1❘⨂ ⟨0❘ + 0.1⋅❘1⟩⨂ ❘0⟩⋅⟨1❘⨂ ⟨1❘ + 0.4⋅❘1⟩⨂ ❘1⟩⋅⟨1❘⨂ ⟨1❘

0.500000000000000

         2      2     2                        2     2                        
0.5⋅⟨0❘0⟩ ⋅⟨1❘1⟩ ⋅1⨂ 1  + 0.1⋅⟨0❘0⟩⋅⟨1❘0⟩⋅⟨1❘1⟩ ⋅1⨂ 1  + 0.4⋅⟨0❘1⟩⋅⟨1❘0⟩⋅⟨1❘1⟩

2     2
 ⋅1⨂ 1 

In [263]:
res=0.5 * g + 0.1 * h
display(res)

display(res.args[0])
display(res.args[1])

res.has(sq.Ket)
print(isinstance(res,sp.Add))
print(isinstance(res,sp.Mul))

res2=res.args[0]
print(isinstance(res2,sp.Add))
print(isinstance(res2,sp.Mul))

0.5⋅❘0⟩ + 0.1⋅❘1⟩

0.5⋅❘0⟩

0.1⋅❘1⟩

True
False
False
True


In [ ]:
g = bKet(0)
h = bKet(1)

TP=sq.TensorProduct
zz=TP(g,g)
zo=TP(g,h)
oo=TP(h,h)
oz=TP(h,g)

display(oz)

ρ = 0.5 * oz*sq.Dagger(oz) + 0.4*oo*sq.Dagger(oo) + 0.1*oz*sq.Dagger(oo)

display(ρ)

k=TP(sq.IdentityOperator(),h)


display(tsimp(sq.Dagger(k)*ρ*k))

display(sq.qapply(sq.Dagger(k)*ρ*k))

del TP

In [ ]:
#tsimp=sq.tensorproduct.tensor_product_simp_Mul

#If an inner product evaluates to one, it drops the dimension
#Works only with products and sums for the moment
#No sanity checks were done
#Hopefully, sympy will get updated with this functionality soon anyway



#def(e):
    

Identity=sq.IdentityOperator()
g=bKet(0)
h=bKet(1)

ρ_g=g*sq.Dagger(g)
ρ_h=h*sq.Dagger(h)

# #sq.qapply(Identity*g); seems to work

# #display(sq.TensorProduct(ρ_g,ρ_h))
# print("g")
# g_=sq.TensorProduct(Identity,g*sq.Dagger(g)); display(g_)
# print("h")
# h_=sq.TensorProduct(Identity,h*sq.Dagger(h)); display(h_)

# print("gh")
# display(sq.Dagger(g_)*h_)

f_=sq.TensorProduct(g,g)
f__=sq.TensorProduct(h,g)

#sq.qapply(sq.Dagger(f_)*g_*f_).doit()

print("inner product of ")
display(f_,f__)
#sq.qapply(sq.Dagger(f_)*f_)
#sq.tensorproduct.tensor_product_simp(sq.Dagger(f_)*f__ + sq.Dagger(f__)*f__)
print("yields these two")
display(sq.Dagger(f_)*f__)
display(sq.qapply(sq.Dagger(f_)*f__).doit())

print("inner product of ")
display(f_,f_)
print("yields these two")
res1=sq.Dagger(f_)*f_
display(sq.Dagger(f_)*f_)
display(sq.qapply(sq.Dagger(f_)*f_).doit())


print("Let us try the inner product of these two vectors with the following operator")
op=f_*sq.Dagger(f_)

display(op)
print("which yields these two")
display(sq.qapply(sq.Dagger(f__)*op*f__))
display(sq.qapply(sq.Dagger(f__)*op*f__).doit())
#sq.qapply(sq.Dagger(f_)*h_*sq.Dagger(f__)).doit()


######So far it works well
print("Consider now the tensor product of two operators")
ρ=sq.TensorProduct(ρ_g,ρ_g)
display(ρ)
print("Their inner product with the ket") 
display(f_)
print("yields the following two")
res=sq.qapply(sq.Dagger(f_)*ρ*f_)
display(res)
display(res.doit())

display(tsimp(res))
print("Testing if the constituents are still instances of orthogonal vectors")
print(isinstance(res.args[0].args[0],bBra))
display(res.args[0].args[0],res.args[1])

print("Testing if the constituents are ... same for the original")
print(isinstance(res1.args[0].args[0],bBra))
display(res1.args[0].args[0],res1.args[1])

#sq.qapply(sq.Dagger(g_)*h_).doit() #*ρ_g

#sq.qapply(sq.Dagger(g)*h).doit()
print("Now we try with this ")
ρ2=sq.TensorProduct(ρ_g,ρ_h)
σ=3*ρ
display(σ)
res2=tsimp(sq.qapply(sq.Dagger(f_)*σ*f_))
display(res2)

#res2.has(sq.TensorProduct)
#isinstance(res2,sq.TensorProduct)
#new_args=list(sq.TensorP)
#sq.TensorProduct(*list(res2.args))


# res3=autoDropDim(res2)
# # for _ in res3.args:
# #     display(_)

# display(res3)

In [ ]:


def tensor_product_simp(e, **hints):
    """Try to simplify and combine TensorProducts.

    """
    if isinstance(e, sp.Add):
        return sp.Add(*[tensor_product_simp(arg) for arg in e.args])
    elif isinstance(e, sp.Mul):
        return tensor_product_simp_Mul(e)
    else:
        return e
    
def tensor_product_simp_Mul(e):
    """Simplify a Mul with TensorProducts.

        >>> e
        AxB*CxD
        >>> tensor_product_simp_Mul(e)
        (A*C)x(B*D)

    """
    
    # TODO: This won't work with Muls that have other composites of
    # TensorProducts, like an Add, Commutator, etc.
    # TODO: This only works for the equivalent of single Qbit gates.
    
    if not isinstance(e, sp.Mul):
        return e
    c_part, nc_part = e.args_cnc()
    n_nc = len(nc_part)
    if n_nc == 0:
        return e
#     elif n_nc == 1:
#         if isinstance(nc_part[0], Pow):
#             return  Mul(*c_part) * tensor_product_simp_Pow(nc_part[0])
#         return e
    elif e.has(sq.TensorProduct):
        current = nc_part[0] #I guess this is the 
#         if not isinstance(current, sq.TensorProduct):
#             if isinstance(current, Pow):
#                 if isinstance(current.base, sq.TensorProduct):
#                     current = tensor_product_simp_Pow(current)
#             else:
#                 raise TypeError('TensorProduct expected, got: %r' % current)
        n_terms = len(current.args)
        new_args = list(current.args)
        for next in nc_part[1:]:
            # TODO: check the hilbert spaces of next and current here.
            if isinstance(next, sq.TensorProduct):
                if n_terms != len(next.args):
                    raise QuantumError(
                        'TensorProducts of different lengths: %r and %r' %
                        (current, next)
                    )
                for i in range(len(new_args)):
                    new_args[i] = new_args[i] * next.args[i]
#             else:
#                 if isinstance(next, Pow):
#                     if isinstance(next.base, sq.TensorProduct):
#                         new_tp = tensor_product_simp_Pow(next)
#                         for i in range(len(new_args)):
#                             if( new_args[i] )
#                             new_args[i] = new_args[i] * new_tp.args[i]
#                     else:
#                         raise TypeError('TensorProduct expected, got: %r' % next)
#                 else:
#                     raise TypeError('TensorProduct expected, got: %r' % next)
            current = next
        return Mul(*c_part) * sq.TensorProduct(*new_args)
    elif e.has(Pow):
        new_args = [ tensor_product_simp_Pow(nc) for nc in nc_part ]
        return tensor_product_simp_Mul(Mul(*c_part) * sq.TensorProduct(*new_args))
    else:
        return e

In [ ]:
glaba=sp.Integer(1)
#isinstance(glaba,sp.number)
#type(glaba)
go=bKet(2)


### Debugging the issue with substitutions

In [ ]:

iKCBS.b[1].subs({iKCBS.a[8]:0})

iKCBS.c
#b=sp.symbols('b0:%d'%10)
#b[0].subs(b[0],10)
# gla=sp.Symbol('x'); gla
# gla.subs(gla,10)

#iKCBS.a[8].subs(iKCBS.a[8],0)

In [ ]:
ket1=bKet(0)
ket2=bKet(1)
kets=(ket1,ket2)
bra=[sq.Dagger(iket) for iket in kets]; bra

### Debugging the issue with _eval_args

In [ ]:
class cls_iKCBS:    
    
    def __init__(s,n):
        cls_iKCBS.n=n
        #j in 0,1,2,3,4
        s.ke = [bKet(j) for j in range(3)]    
        s.br = [sq.Dagger(s.ke[j]) for j in range(3)]
        s.πs = []
        s.matπs = [] 
        #poor man's implementation
        for j in range(5):
            s.πs.append(s.proj(j,n))            
            s.matπs.append(s.matRep(s.proj(j,n)))
        s.opT = s.ke[0]*s.br[2] + s.ke[1]*s.br[0] + s.ke[2]*s.br[1]
        s.matT=s.matRep(s.opT)

    #this method is now obsolete    
    def matRep(s,op):
        n=3 #s.__class__.n
        matop = sp.Matrix(np.zeros((n,n)))        
        for i in range(n):
            for j in range(n):
                matop[i,j] = sq.qapply(sq.Dagger(s.ke[i])*op*s.ke[j]).doit() 
        return matop
        
    def proj(s,j,n):
        N=sp.Integer(n)
        J=sp.Integer(j)
        #print(N,j)
        one=sp.Integer(1)
        #print(one)
        α1=sp.cos((sp.pi)/N)/(one+sp.cos(sp.pi/N)) #verified
        #a1=sp.cos(sp.pi/N)
        #
        #print(α1)
        
        α2=one-α1 #verified
        α3=J * sp.pi * (N-one)/N #verified
        
        vec = sp.sqrt(α1)*s.ke[0] + sp.sqrt(α2)*sp.sin(α3)*s.ke[1] + sp.sqrt(α2)*sp.cos(α3)*s.ke[2]
        
        projector = vec*sq.Dagger(vec)
           
        #matrixprojector = sp.Matrix([0,0])
        return projector
    
    
    def find_coeff_matT(s):
        cls=s.__class__
        a=sp.symbols('a0:%d'%(cls.n*2))
        LHS=a[0]*s.matπs[0]
        for i in range(1,cls.n):
            LHS=LHS+a[i]*s.matπs[i]
        for i in range(cls.n,2*cls.n):
            LHS=LHS+a[i]*s.matπs[i%cls.n]*s.matπs[(i+2)%cls.n]
            
        RHS=s.matT
        return sp.solve(sp.N(LHS-RHS),a)
        #return sp.solve(a[0]-a[2],a)
        
        #print(LHS)
        #for i in range
        

    '''
       def vec(self,j,n):    
           N=sp.Integer(n)
           J=sp.Integer(j)
           #print(N,j)
           one=sp.Integer(1)
           #print(one)
           α1=sp.cos((sp.pi)/N)/(one+sp.cos(sp.pi/N)) #verified
           #a1=sp.cos(sp.pi/N)
           #
           #print(α1)
        
           α2=one-α1 #verified
           α3=J * sp.pi * (N-one)/N #verified
        
           vec = sp.sqrt(α1)*self.k0 + sp.sqrt(α2)*sp.sin(α3)*self.k1 + sp.sqrt(α2)*sp.cos(α3)*self.k2
        
           #projector = vec*sq.Dagger(vec)
           return vec
    '''

In [ ]:

iKCBS=cls_iKCBS(5)
#to access the vectors 

# #uncomment to test
# sq.qapply(sq.qapply(iKCBS.πs[0]*iKCBS.πs[1]))
# #iKCBS.πs[1]

# sp.N(iKCBS.matπs[2])

# iKCBS.find_coeff_matT()

In [ ]:
iKCBS.find_coeff_opT()

### Debugging

In [ ]:
sq.qapply(sq.Dagger(iKCBS.πs[0])*iKCBS.πs[1]);

#sq.represent(sq.qapply(sq.Dagger(iKCBS.πs[0])*iKCBS.πs[1]),basis=iKCBS.ke)

In [ ]:
#older version without lists
class cls_iKCBS:
    k0=bKet(0)    
    k1=bKet(1)
    k2=bKet(2)
    
    b0=sq.Dagger(k0)
    b1=sq.Dagger(k1)
    b2=sq.Dagger(k2)
    
    πs=[]
    
    def __init__(self,n):
        #j in 0,1,2,3,4
        for j in range(5):
            self.πs.append(self.proj(j,n))
    
    def proj(self,j,n):
        N=sp.Integer(n)
        J=sp.Integer(j)
        #print(N,j)
        one=sp.Integer(1)
        #print(one)
        α1=sp.cos((sp.pi)/N)/(one+sp.cos(sp.pi/N)) #verified
        #a1=sp.cos(sp.pi/N)
        #
        #print(α1)
        
        α2=one-α1 #verified
        α3=J * sp.pi * (N-one)/N #verified
        
        vec = sp.sqrt(α1)*self.k0 + sp.sqrt(α2)*sp.sin(α3)*self.k1 + sp.sqrt(α2)*sp.cos(α3)*self.k2
        
        projector = vec*sq.Dagger(vec)
        return projector
    '''
       def vec(self,j,n):    
           N=sp.Integer(n)
           J=sp.Integer(j)
           #print(N,j)
           one=sp.Integer(1)
           #print(one)
           α1=sp.cos((sp.pi)/N)/(one+sp.cos(sp.pi/N)) #verified
           #a1=sp.cos(sp.pi/N)
           #
           #print(α1)
        
           α2=one-α1 #verified
           α3=J * sp.pi * (N-one)/N #verified
        
           vec = sp.sqrt(α1)*self.k0 + sp.sqrt(α2)*sp.sin(α3)*self.k1 + sp.sqrt(α2)*sp.cos(α3)*self.k2
        
           #projector = vec*sq.Dagger(vec)
           return vec
    '''

In [ ]:
iKCBS=cls_iKCBS()
sq.qapply(iKCBS.proj(1,5)).doit();

In [ ]:
sp.simplify(sq.qapply(sq.Dagger(iKCBS.k0)*iKCBS.proj(1,5)*iKCBS.proj(2,5)*iKCBS.k0).doit())
#sp.N(sq.qapply(sq.Dagger(iKCBS.k0)*iKCBS.proj(1,5)*iKCBS.proj(2,5)*iKCBS.k0).doit())
#sq.qapply(iKCBS.proj(1,5)*iKCBS.proj(2,5)).doit()

In [ ]:
(sq.Dagger(iKCBS.k0)*iKCBS.k0).doit()

In [ ]:
sp.N(sq.qapply(sq.Dagger(iKCBS.vec(1,5))*iKCBS.vec(1,5)).doit(),5000)

In [ ]:
sp.N(sp.simplify(sq.qapply(sq.Dagger(iKCBS.vec(4,5))*iKCBS.vec(3,5)).doit()))

In [ ]:
sp.N(sq.qapply(sq.Dagger(iKCBS.πs[0])*iKCBS.πs[1]))

## Translation Operator using $\Pi$s

See for the original implementation Self-testing_KCBS_fidelity_bounds

In [ ]:

a=sp.symbols('a0:%d'%(cls_iKCBS.n*2))
a[0]
c=0
c=c+a[0]; c


In [ ]:
a=sp.Matrix(np.zeros((2,2)))
a[1,1]=0;a


In [ ]:
a=sqq.IntQubit(2);a

b=sqq.IntQubit(1);
sq.represent(b);
sq.represent(a)

# SDP part (SymPy meets CvxPy)

In [ ]:
N=5

#Ψ=[sp.symbols('Ψ')]
I=[sp.Integer(1)]
A=list(sp.symbols('A_0:%d'%N,commutative=False))
# display(I)
# display(A)

In [ ]:
objective = A[0] + A[1] + A[2] + A[3] #+ A[4]

display(objective);

In [ ]:


L1=A #appends the lists
#print(L1)

L2=[]
for let1 in L1:
    for let2 in L1:
        L2.append(let1*let2)


#These will help simplify A^3 to A when needed
lDepth=2
sqDicts={A[i]**2:A[i] for i in range(len(L1))}


L = I + L1 + L2
L_ = L1 + L2
#print(L)

In [ ]:
#symbol to index dictionary
sTi={}

for i in range(len(L)):
    sTi[L[i]]=i

#print(sTi)

In [ ]:
G = nx.Graph()

#node2 in G[node1] #yes if they are connected by an edge

G = nx.Graph()
G.add_nodes_from(L1)
G.add_edges_from([[L1[i],L1[(i+1)%(N)]] for i in range(N)])

#create a dictionary to simplify expressions later which sets projector products to zero
gDicts={}
for l1 in L1:
    for l2 in L1:
        if l1 in G[l2]:
            gDicts.update({l1*l2:sp.Integer(0)})


%matplotlib inline 
nx.draw(G, with_labels=True)

## Zero list (useless?)

In [ ]:
zeroDict={}
zeroList=[]
for _ in L1:
    for _2 in L1:
        if _ in G[_2]: #share an edge
            zeroDict[_*_2]=0
            zeroList.append(_*_2)

#print(zeroDict);

## Creating the matrix of symbols (just for show; useless)

In [ ]:
#creating the matrix

matList=[]

for _ in L:
    matList.append([_*_2 for _2 in L])

#print(matList)

sp.Matrix(matList);

## Sympy says hello to Cvxpy

In [ ]:
#L is the alphabet, to wit: it is the list of letters
#
X = cp.Variable((len(L),len(L)),symmetric=True)

lI = I[0]

#>> is for matrix inequality
constraints = [X >> 0,X[sTi[lI],sTi[lI]]==1]

#objective
#y = objective.subs({A[0]:X[sTi[A[0]]],A[1]:X[sTi[A[1]]]})
#y = objective.subs({A[0]:sTi[A[0]],A[1]:sTi[A[1]]})


# objective function evaluated (loop over product of all letters)
y = 0

termsUsed = []
for l1 in L:
    for l2 in L:
        if l1 != lI or l2 != lI: #this is because if we ask for the coefficient of 1, things get messed up
            term = l1*l2
            
            #the same term can be represented by many elements in the matrix
            #we satisfy ourselves with picking just one of them
            if not(term in termsUsed):
                termsUsed.append(term)                
                eX = X[sTi[l1]][sTi[l2]] #picks the corresponding element from the variable matrix of the SDP library
                #print(coeff)
                #print(l1,l2)
                coeff = sp.N(objective.coeff(l1*l2)) #evaluates the coff of l1*l2 in the objective
                y += coeff*eX


#Constraints from the graph (these come from single letters)        
for l1 in L1:
    for l2 in L1:
        eX = X[sTi[l1]][sTi[l2]] #picks the corresponding element from the variable matrix of the SDP library
        if l1 in G[l2]:
            constraints += [eX == 0]
        
        #print(coeff,sTi[l1])
        #display(l1)

#constraints from the fact that Π^2=Π
# for l in L1:
#     constraints += [ X[sTi[lI]][sTi[l]] == X[sTi[l]][sTi[l]] ]
#     #constraints to match with experiment

dictTerms = {}
#lettersUsed = []
for l1 in L:
    for l2 in L:
        term = l1*l2

        
        tS=[term]
        #display(term)
        for i in range(lDepth+1):
            tS.append(tS[-1].subs(sqDicts))
        simTerm=tS[-1]
        #display(simTerm)

        if not(simTerm in dictTerms):
            dictTerms.update({simTerm:[l1,l2]})            
        else:
            l1_=dictTerms[simTerm][0]
            l2_=dictTerms[simTerm][1]
            #print(l1_,l2_)            
            #print(sTi[l1_],sTi[l2_])
            
            #print(l1,l2)            
            #print(sTi[l1],sTi[l2])
            
            #constraints += [ X[sTi[l1]][sTi[l2]] == X[sTi[l1_]][sTi[l2_]] ] 
            # print(X[sTi[l1]][sTi[l2]])
            # print(X[sTi[l1_]][sTi[l2_]])
            constraints += [ X[sTi[l1]][sTi[l2]] == X[sTi[l1_]][sTi[l2_]] ]

        #Orthogonality from the graph
        if simTerm.subs(gDicts)==sp.Integer(0):
            constraints += [ X[sTi[l1]][sTi[l2]] == 0 ]
        del tS
# display(y)

# display(objective.args[0])
# display(objective.args[1])


#expr=A[0]*A[0]*A[0]*A[1]*A[0]*A[1]

    



#display(tS[-1])


# powDicts={}
# for i in range(len(L1)):
#     powDicts.update({A[i]**k:A[i] for k in range(2,lDepth+4)[::-1]})


#expr.subs(powDicts)

# display(expr.subs(sqDicts))
# display(expr.subs(powDicts).subs(powDicts))

#expr
    
#dict={A[i]**k:A[i] for k in range(lDepth)}

#print(powDicts)

#result=expr.subs({A[0]*A[0]:A[1]}); result
#result=sp.refine(expr,)

In [ ]:
prob = cp.Problem(cp.Maximize(y),constraints)
#prob.solve()
#prob.solve(solver=cp.CVXOPT,verbose=True,**{'':ldl,'refinement':20,'show_progress':True,'feastol':10e-10,'reltol':10e-10,'abstol':10e-10})
prob.solve(solver=cp.CVXOPT,verbose=True)

In [ ]:
import cvxopt as co



## Inputs